<b><font size=20, color='#A020F0'>Pandas

Hannah Zanowski<br>
10/3/22<br>

#### <span style="color:green">Learning Goals</span>
By the end of this notebook you will
1. Become familiar with the basic data structures in Pandas and how to create them
2. Practice reading in data and doing computations on dataframes

#### Resources
[Pandas website](https://pandas.pydata.org/)<br>
[Pandas Documentation](https://pandas.pydata.org/docs/)<br>
[Pandas API reference](https://pandas.pydata.org/docs/reference/index.html#)<br>

# A little about pandas

Pandas is a glorious library for working with tabular data. It relies on high-level data structures and comes with a huge set of useful analysis tools, including easily reading in .txt and .csv files. Pandas uses label-based indexing that makes working with data much more streamlined and easier to understand.

Let's begin by importing pandas (and a few of our other favorites):

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

---

### 1. Creating a pandas series
[Series](https://pandas.pydata.org/docs/reference/api/pandas.Series.html) objects are a core data structure in pandas. A series is much like a 1D numpy ndarray, except that there is a labeled index associated with the values in the series. Like all things python, there are many ways to [create a pandas series](https://pandas.pydata.org/pandas-docs/stable/user_guide/dsintro.html#series)

In [ ]:
#Create a pandas series from scratch
animal_counts=pd.Series(data=[10,8,22,1,17,6,3,0], 
                      index=['Dog','Cat','Bird','Alligator',
                             'Giraffe','Lion','Bear','Penguin'])

In [ ]:
animal_counts

### Working with series

#### Accessing the data
You can access series data by using `.loc[]` or `.iloc[]`. The first allows you to retrieve a value based on the named index, and the second allows you to retrieve a value based on an integer index (like we use in numpy arrays)

In [ ]:
animal_counts.loc['Cat']

In [ ]:
animal_counts.iloc[1]

If you want to access only the data:

In [ ]:
animal_counts.values

If you want to access only the index:

In [ ]:
animal_counts.index

#### Doing computations
You can apply most numpy functions to pandas series without any issue. The best part is that your index will be preserved!

In [ ]:
animal_counts.shape

In [ ]:
(animal_counts-np.mean(animal_counts))/np.std(animal_counts,ddof=1)

---

### 2. Creating a pandas dataframe
[DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html#pandas-dataframe) objects are another core data structure in pandas. A dataframe is basically a 2D container for a collection of series with the same index. You can do computations across the columns of a dataframe, and you can think of it like a spreadsheet or any other table of data. Like series, there are many ways to [create a pandas dataframe](https://pandas.pydata.org/docs/user_guide/dsintro.html#dataframe).

As an example, let's create a pandas dataframe using a dictionary:

In [ ]:
class_data = {'quarter':[1,3,4],
        'year':[2015,2014,2019],
        'enrollment':[200,np.nan,85],
        'prev_enrollment':[150,10,102]}
df = pd.DataFrame(class_data, index=['AOS100', 'AOS742', 'AOS384'])
df

You can extract summary information about a pandas dataframe with `.info()`:

In [ ]:
df.info()

You can also extract summary statistics from a dataframe with `.describe()`:

In [ ]:
df.describe()

---

### 3. Accessing data in a pandas dataframe
You can access data in a dataframe with index labels or index numbers like we used before, and you can access entire columns with column names

In [ ]:
df.loc['AOS100']

In [ ]:
df['enrollment']
#or df.enrollment

In [ ]:
#Access more than one column or row:
df[['quarter','enrollment']]

#### Filtering based on boolean operators
You can filter your datasets based on conditions

In [ ]:
df[df.prev_enrollment>50]

#### Filtering with .where()
Pandas' [where()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.where.html) is another really useful tool for filtering, but <b><font color='red'>NOTE:</font></b> It only works when all of the columns are the same datatype!

In [ ]:
df.where(df.quarter<2)

---

### 4. Creating new columns, merging data, and reindexing

In [ ]:
#Create a new column in our dataframe
df['enrollment_change'] = (df.enrollment-df.prev_enrollment)/df.prev_enrollment
df

In [ ]:
#Create a new column separately and merge into our dataframe
instructor=pd.Series(['Rick', 'Morty', None ],
                     index=['AOS100', 'AOS742', 'AOS384'],
                     name='instructor')
# returns a new DataFrame
df=df.join(instructor)

In [ ]:
#Add another index (in our case another class) to the dataframe
df=df.reindex(index=['AOS100', 'AOS742', 'AOS384', 'AOS121'])
df

### 5. Setting data in a dataframe
You can use [at[]](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.at.html) or [iat[]](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.iat.html) to set values in a dataframe:

In [ ]:
#Assign data to some of the columns
df.at['AOS121','quarter']=2
df

You can also set data in place using conditions:

In [ ]:
#Create a subset of the dataframe to work on for this example
df_new=df.drop(columns='instructor')

#Make a condition based on enrollment
df_new[df_new.enrollment>50]=-50

---

### 6. Reading in data
Pandas can [read in all sorts](https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html) of tabular data (csv, excel, even stuff from your local clipboard!!). Let's use pandas to read in our Niño3.4 data from the first in-class notebook. If you don't remember where it is just go ahead and download it again:

```bash
!curl -O https://www.cpc.ncep.noaa.gov/products/analysis_monitoring/ensostuff/detrend.nino34.ascii.txt
```

In [ ]:
df_n34=pd.read_csv('detrend.nino34.ascii.txt',sep='\s+')
df_n34.head() #default is to show only 5 rows; you can change this by providing a different argument to the function

In [ ]:
df_n34.iloc[856]

In the above case we didn't set an index (so it's just a number), but we could set whatever index we want, including one of the data columns. Let's create our own index called 'date' using pandas' [to_datetime()](https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html) function. `to_datetime()` can make datetime objects out of columns of a pandas dataframe, but the columns need to be named 'year', 'month' etc for pandas to be able to recognize them as years and months. So let's rename our 'YR' and 'MON' dataframe columns to reflect this:

In [ ]:
df_n34=df_n34.rename(columns={'YR':'year','MON':'month'})

In [ ]:
df_n34.columns

Now let's create a new datetime index and use the date as the index:

In [ ]:
#Make the new index
new_index=pd.to_datetime(df_n34[['year', 'month']].assign(days=np.ones(len(df_n34.ANOM.values))))
new_index

In [ ]:
#Set the index to be new_index
df_n34=df_n34.set_index(new_index)
df_n34

We can now access values by the date index that we created:

In [ ]:
df_n34.loc['1997']

We aren't going to use the TOTAL and ClimAdjust columns here, so let's get rid of them using [drop()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html):

In [ ]:
df_n34=df_n34.drop(columns=['TOTAL','ClimAdjust'])
df_n34

---

### 6. Doing computations on your dataframe
Let's revisit some of what we did in the first in-class notebook to (hopefully) show you how doing computations in pandas is much more intuitive than using unlabeled numpy arrays!

Now let's get the start and end dates of df_n34:

In [ ]:
#print start and end dates of the dataframe
print(df_n34.index[0])
print(df_n34.index[-1])

#or 
#df_n34.info()

In [ ]:
#Get the subset of Niño3.4 SST anomaly data that covers the same date range as the SOI data
df_n34.loc['1951-01':'2021-12']

#### Counting
You can count the number of non-nan elements in the columns of a DataFrame with [count()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.count.html) and you can retrieve counts of unique items in a pandas Series with [value_counts()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.value_counts.html)

In [ ]:
df_n34.count() #apply to dataframe

In [ ]:
df_n34.month.value_counts() #applies to a Series object

#### Groupby

Pandas' [groupby](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html) functionality is also really powerful. It allows you to group data based on a condition that you provide. You can then do computations on the resulting groups. A common use of groupby is grouping by time like we'll do below, but you can use groupby in many other ways as well, as long as it makes sense for your data!

As an example, we'll use `groupby()` to create a monthly climatology for our Niño3.4 SST anomalies from Jan 2000 to Dec 2021:

In [ ]:
#First get the dates we want:
n34_2000_2021=df_n34.ANOM.loc['2000-01':'2021-12']

#Then use groupby to group the same months together, and then take the mean over all of the years
n34_climo=n34_2000_2021.groupby(n34_2000_2021.index.month).mean('year')

n34_climo

#### Resampling

Pandas' [resample()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.resample.html) function is useful for resampling time series. <b>For it to work properly, your dataframe must have a datetime-like index!</b>

For this example, we'll downsample the Niño3.4 SST anomalies from monthly output to annual output (note that in this case you could just use groupby to get the same result):

In [ ]:
n34_ann=df_n34.ANOM.resample('AS').mean() #'AS' means annual
n34_ann.head()

### 7. Plotting data
Pandas has a number of basic [built-in plotting options](https://pandas.pydata.org/pandas-docs/stable/user_guide/visualization.html) for visualizing your data. You can also plot pandas data using matplotlib, etc.

Let's start by plotting the Niño3.4 SST anomalies using built-in plotting commands:

In [ ]:
#Plot Niño3.4 SST anomalies using built-in plotting
df_n34['ANOM'].plot(title='Niño3.4 SST anomalies',xlabel='Time (years)',ylabel='SST anomaly ($\degree$C)');

Now let's plot the anomalies using matplotlib. Notice how we don't need to create our own time axis like we had to in the first notebook wwhen we were using raw numpy arrays! This is because we created a `DatetimeIndex` for our data.

In [ ]:
#Plot Niño3.4 SST anomalies using matplotlib
fig,ax=plt.subplots()
fig.set_size_inches(10,3)
ax.plot(df_n34['ANOM'],color='0.5',lw=1)
plt.title('Niño3.4 SST anomalies')
plt.xlabel('Time (years)')
plt.ylabel('SST anomaly ($\degree$C)');
ax.set_xlim([datetime.date(1950,1,1),datetime.date(2020,1,1)])

#plot the resampled data as well
plt.plot(n34_ann,color='k',lw=2)

Let's make a histogram of the monthly Niño3.4 SST anomalies:

In [ ]:
df_n34.ANOM.hist(bins=np.arange(-2.25,2.26,0.5),color='navy',ec='goldenrod',grid=False);
#bins here are the bin edges!